In [127]:
using Random
using LinearAlgebra
using SparseArrays
using PyCall
using Statistics

In [128]:
L = 2;
Number_Of_Noise = 4*L^2-6*L+13-3;
Random.seed!(2022)
NOISE = 2*rand(Float64,Number_Of_Noise).-1;

In [129]:
#Rx(theta) = exp(-1im*(theta*[-1 1;1 -1]/2));
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];
Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);

In [130]:
int(x) = floor(Int,x)

int (generic function with 1 method)

In [149]:
#Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];#

#round.(-exp(-1im*pi*([1 1;1 1]/2)); digits = 3)

Ry(theta) = [cos(theta/2) -sin(theta/2) ; sin(theta/2) cos(theta/2)];

Pauli_X = [0 1;1 0];
Pauli_Y = [1 -1im;1im 0];
Pauli_Z = [1 0;0 -1];
X = [0 1;1 0];
I2 = [1 0; 0 1];
Z = [1 0;0 -1];
H = (1/sqrt(2))*[1 1;1 -1]
Hadamard(noise) = exp(-1im*(pi/2+noise)*(I2-H)) #Ry(pi/2+noise)*Pauli_Z;
CX(noise) = exp(-1im*(pi/2+noise)*(X-I2));

In [150]:
Had = Hadamard(0.01)

2×2 Matrix{ComplexF64}:
 0.707136+0.00292874im   0.707036-0.0070706im
 0.707036-0.0070706im   -0.706936+0.0170699im

In [135]:
"""

Following function takes a 2x2 matrix (Gate) and qubit position (Qubit) and
returns the resultant matrix.

For example, the matrix for the gate U acting on the 3-rd qubit for N=5
qubit system is given by   I (x) I (x) U (x) I (x) I; where (x) is the
tensor product.

"""

function Matrix_Gate(Gate, Qubit) # Previously known as multi qubit gate.
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:L
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:L
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end;

In [136]:
Identity(dimension) = 1* Matrix(I, dimension, dimension);

In [137]:
"""

The following function returns a controlled U gate matrix.

Input  : c (integer), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubit c and target qubit t.

"""

function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;

In [138]:
"""

The following returns a multicontrolled U gate matrix.

Input  : c (list), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubits c and target qubit t.

"""

function MCU(c,t,U)
    
    p0 = fill("I", L)
    p1 = fill("I", L)

    
    if typeof(c) == Int64
        p0[c] = "PI_1"
        p1[t] = "PI_1"
        
    else
        for i in c
            p0[i] = "PI_1"
            p1[i] = "PI_1"
        end
    end
    
    p0[t] = "I"
    p1[t] = "U"

    
    I = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])
    X = sparse([0 1;1 0])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2
     
    Matrices = Dict("I" => I,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
             
    # The identity in the following line needs to be replaced.
    return Identity(2^L) - PI_0_matrix + PI_1_matrix     
end;

In [139]:
using PyCall
py"""
import numpy
import numpy.linalg
def adjoint(psi):
    return psi.conjugate().transpose()
def psi_to_rho(psi):
    return numpy.outer(psi,psi.conjugate())
def exp_val(psi, op):
    return numpy.real(numpy.dot(adjoint(psi),op.dot(psi)))
def norm_sq(psi):
    return numpy.real(numpy.dot(adjoint(psi),psi))
def normalize(psi,tol=1e-9):
    ns=norm_sq(psi)**0.5
    if ns < tol:
        raise ValueError
    return psi/ns
def is_herm(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M-adjoint(M)
    return max(numpy.abs(diff.flatten())) < tol
def is_unitary(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M.dot(adjoint(M))-numpy.identity((M.shape[0]))
    return max(numpy.abs(diff.flatten())) < tol
def eigu(U,tol=1e-9):
    (E_1,V_1)=numpy.linalg.eigh(U+adjoint(U))
    U_1=adjoint(V_1).dot(U).dot(V_1)
    H_1=adjoint(V_1).dot(U+adjoint(U)).dot(V_1)
    non_diag_lst=[]
    j=0
    while j < U_1.shape[0]:
        k=0
        while k < U_1.shape[0]:
            if j!=k and abs(U_1[j,k]) > tol:
                if j not in non_diag_lst:
                    non_diag_lst.append(j)
                if k not in non_diag_lst:
                    non_diag_lst.append(k)
            k+=1
        j+=1
    if len(non_diag_lst) > 0:
        non_diag_lst=numpy.sort(numpy.array(non_diag_lst))
        U_1_cut=U_1[non_diag_lst,:][:,non_diag_lst]
        (E_2_cut,V_2_cut)=numpy.linalg.eigh(1.j*(U_1_cut-adjoint(U_1_cut)))
        V_2=numpy.identity((U.shape[0]),dtype=V_2_cut.dtype)
        for j in range(len(non_diag_lst)):
            V_2[non_diag_lst[j],non_diag_lst]=V_2_cut[j,:]
        V_1=V_1.dot(V_2)
        U_1=adjoint(V_2).dot(U_1).dot(V_2)
    # Sort by phase
    U_1=numpy.diag(U_1)
    inds=numpy.argsort(numpy.imag(numpy.log(U_1)))
    return (U_1[inds],V_1[:,inds]) # = (U_d,V) s.t. U=V*U_d*V^\dagger
"""

py"""
f = open('exact_G_eigenvalues'+'.txt', 'w')
def Write_file(Eigenvalue):
    f = open('exact_G_eigenvalues'+'.txt', 'a')
    f.write(str(Eigenvalue) +'\n')
"""

In [140]:
function MyEigenvalues(DELTA)
    List_of_H = [];
    Grover_matrix = sparse(Identity(2^L));
    Noise_Used = [];
    Noise_Counter = 1;
    
    #= U0 matrix. =#
    U0_XHL_Gates = []
    for i = 1:L-1
        push!(U0_XHL_Gates,["X",i])
    end    
    push!(U0_XHL_Gates,["H",L])

    U0_XHR_Gates = []
    for i = 1:L-1
        push!(U0_XHR_Gates,["X",i])
    end
    push!(U0_XHR_Gates,["H",L])
    
    MCX = sparse(Identity(2^L));
    
    XHL_Matrix = sparse(Identity(2^L))
    for i in U0_XHL_Gates
        if i[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), i[2]) 
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["H", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
        elseif i[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),i[2])#Rx(pi+DELTA*epsilon)
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["X", DELTA*epsilon,i[2]])
            Noise_Counter += 1   
        end
    end
    
    #= Constructing the multicontrolled Toffoli gate. =# 
    # C_1.
    for i = 1:L-2
        for j = 1:i
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            push!(List_of_H,[pi/2^j+DELTA*epsilon, L-i, L-i+j])
            Noise_Counter += 1
            
        end
    end

    # C_2.
    for i = 2:L

        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        push!(List_of_H, [pi/2^(i-2)+DELTA*epsilon, 1, i])
        Noise_Counter += 1
        
    end

    # C3 = - C1.
    for i = L-2:-1:1
        for j = i:-1:1
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            push!(List_of_H,[-pi/2^j+DELTA*epsilon, L-i, L-i+j])
            Noise_Counter += 1
        end
    end

    # C_4.
    for i = 1:L-3
        for j = 1:i
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            push!(List_of_H,[pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1
        end    
    end

    # C_5.
    for i = 2:L-1
        
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((-pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        push!(List_of_H,[-pi/2^(i-2)+DELTA*epsilon,1, i])
        Noise_Counter += 1
        
    end

    # C6 = - C4.
    for i = L-3:-1:1
        for j = i:-1:1
         
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1            
        end    
    end

    XHR_Matrix = sparse(Identity(2^L))
    for j in U0_XHR_Gates
        if j[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), j[2])
            XHR_Matrix = XHR_Matrix*CRX
            push!(List_of_H,["H", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
            
        elseif j[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),j[2])
            XHR_Matrix = XHR_Matrix*CRX
            push!(List_of_H,["X", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
        end
    end

    U0_matrix = sparse(XHL_Matrix*MCX*XHR_Matrix)    

    
    #= Ux matrix. =#
    Ux_XHL_Gates = []
    for i = 1:L-1
        push!(Ux_XHL_Gates,["H",i])
    end    
    for i = 1:L-1
        push!(Ux_XHL_Gates,["X",i])
    end  

    Ux_XHR_Gates = []
    for i = 1:L-1
        push!(Ux_XHR_Gates,["X",i])
    end    
    for i = 1:L-1
        push!(Ux_XHR_Gates,["H",i])
    end
    

    # Creating an empty matrix to store the Ux matrix.
    MCX = sparse(Identity(2^L));
    
    XHL_Matrix = sparse(Identity(2^L))
    for i in Ux_XHL_Gates
        
        if i[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), i[2])
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["H", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
            
        elseif i[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),i[2])
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["X", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
            
        end
    end
    
    #= Contructing the multicontrolled Toffoli gate. =#
    # C_1.
    for i = 1:L-2
        for j = 1:i
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            push!(List_of_H,[pi/2^j+DELTA*epsilon,L-i, L-i+j])
            Noise_Counter += 1          
        end
    end

    # C_2.
    for i = 2:L
        
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        push!(List_of_H,[pi/2^(i-2)+DELTA*epsilon, 1, i])
        Noise_Counter += 1
        
    end

    # C3 = - C1.
    for i = L-2:-1:1
        for j = i:-1:1
      
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i, L-i+j])
            Noise_Counter += 1
        end
    end

    # C_4.
    for i = 1:L-3
        for j = 1:i
         
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            push!(List_of_H,[pi/2^j+DELTA*epsilon, L-i-1, L-i-1+j])
            Noise_Counter += 1
        end    
    end

    # C_5.
    for i = 2:L-1
      
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((-pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        push!(List_of_H,[-pi/2^(i-2)+DELTA*epsilon,1, i])
        Noise_Counter += 1
        
    end

    # C6 = - C4.
    for i = L-3:-1:1
        for j = i:-1:1
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1           
        end    
    end


    XHR_Matrix = sparse(Identity(2^L))
    for j in Ux_XHR_Gates
        if j[1] == "H"          
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), j[2]) 
            XHR_Matrix = XHR_Matrix*CRX
            push!(List_of_H,["H", DELTA*epsilon,j[2]])
            Noise_Counter += 1          
        elseif j[1] == "X"         
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),j[2])
            XHR_Matrix = XHR_Matrix*CRX
            push!(List_of_H,["X", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
        end
    end
    
    Ux_matrix = sparse(XHL_Matrix*MCX*XHR_Matrix)   
    
    Grover_matrix = collect(U0_matrix)*collect(Ux_matrix)
    
    #= Diagonalizing Grover for exact eigenvalues. =#
    EIGU = py"eigu"(Grover_matrix)
    E_exact = real(1im*log.(EIGU[1])); # Eigenvalue.   
    E_exact = E_exact[2:2^L-1]; #= Neglecting the two special states. =#
    

    #= Hs_list will contain all the gates H for the corresponding CRX,Hadamard and X in List_of_H. =#
    Hs_list = []
    for i in List_of_H
        
        if i[1] == "H"
            Noise = i[2] # delta*epsilon.
            Qubit = i[3] # qubit.
            Unitary = Matrix_Gate(I2-Hadamard(Noise),Qubit) #= H_had = I2-Had. =#
            push!(Hs_list,[Unitary,Noise])
            
        elseif i[1] == "X"
            Noise = i[2] # delta*epsilon.
            Qubit = i[3] # qubit.
            Unitary = Matrix_Gate(X-I2,Qubit) #= H_X = X-I2. =#
            push!(Hs_list,[Unitary,Noise])
            
        else #= [noise,control,qubit]. =#
            Noise = i[1]
            Control_Qubit = int(i[2])
            Target_Qubit = int(i[3])
            #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
            Matrices = Dict("I" => I2,"U" => (I2+X)/2, "PI_1" => (I2-Z)/2)
            p = fill("I", L)
            p[Control_Qubit] = "PI_1"
            p[Target_Qubit] = "U"
            Unitary = Matrices[p[1]]
            for i = 2:L
                Unitary = kron(Unitary,Matrices[p[i]])
            end                     
            push!(Hs_list,[Unitary,Noise])
        end 
    end
    
    HL = List_of_H;
    
    function kth_term(k)
        #= k^{th} term of the effective Hamiltonian. =#
        p = Identity(2^L);
        q = Identity(2^L);
        for i = k:length(HL)-1
            #= LHS =#
            x1 = HL[length(Hs_list)-i+k][1]
            h_i = Hs_list[length(Hs_list)-i+k][1] # The unitary.
            if x1 == "H"
                noise = Hs_list[length(Hs_list)-i+k][2] # change
                #p = p*exp(-1im*(pi/2)*collect(h_i))
                p = p*collect(h_i)
                
            elseif x1 == "X"
                noise = Hs_list[length(Hs_list)-i+k][2] # change
                p = p*exp(-1im*(pi/2+noise)*collect(h_i))
                
            else
                noise = Hs_list[length(Hs_list)-i+k][2]
                p = p*exp(-1im*noise*collect(h_i))
                
            end  
            #= RHS =#
            x1 = HL[i+1][1]
            h_i = Hs_list[i+1][1]
            if x1 == "H"
                noise = Hs_list[i+1][2]
                q = q*exp(1im*(pi/2)*collect(h_i))
                
            elseif x1 == "X"
                noise = Hs_list[i+1][2]
                q = q*exp(1im*(pi/2+noise)*collect(h_i))
                
            else
                noise = Hs_list[i+1][2]
                q = q*exp(1im*noise*collect(h_i))
            end         
        end
        return p*collect(Hs_list[k][1])*q  
    end
    
    #= The following loop sums over all epsilon to get H_eff. =#
    h_eff = zeros(2^L,2^L);
    for i = 1:length(HL)
        h_eff += Noise_Used[i]*kth_term(i)
    end
    heff = DELTA*h_eff;
    E_eff = eigen(heff[3:2^L,3:2^L]).values
    E_eff = sort(real(E_eff),rev = true)    
    return E_eff,E_exact    
end;

In [141]:
py"""
f = open('eigenvalues_data'+'.txt', 'w')
def Write_file2(delta, effective, exact):
    f = open('eigenvalues_data'+'.txt', 'a')
    f.write(str(delta) + '\t' + str(effective)+ '\t' + str(exact) +'\n')
"""

In [142]:
Num = 80;
for i = 1:Num
    delta = 0.3*(i/Num)
    EE = MyEigenvalues(delta);
    effective = EE[1]
    exact = EE[2]
    for j = 1:2^L-2
        py"Write_file2"(delta,effective[j],exact[j])
    end
end